# Wikipics async demo

## Referências sobre `asyncio` no Jupyter Notebook

* [IPython 7.0, Async REPL (from 2018)](https://blog.jupyter.org/ipython-7-0-async-repl-a35ce050f7f7)
* [Supporting asyncio.get_event_loop().run_until_complete() in repls](https://discuss.python.org/t/supporting-asyncio-get-event-loop-run-until-complete-in-repls/5573/9)
* [Running Asynchronous Code in Jupyter Notebooks: Managing Event Loops](https://medium.com/p/b9696a596ce4) (paywall, não vale o custo)

In [1]:
from pathlib import Path
from typing import NamedTuple

from IPython.display import display
from ipywidgets.widgets import Image, Layout
import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

Função que usamos na demonstração com threads e processos, denominada `fetch` em 
[wikipics-demo](wikipics-demo.ipynb).
Reproduzi aqui para comparar com a versão assíncrona de `fetch`, logo abaixo.

```python
def fetch_blocking(url) -> ImageRecord:
    resp = httpx.get(url)  # thread blocked here
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))
```

In [2]:
from httpx import AsyncClient

async def fetch(client:AsyncClient, url:str) -> ImageRecord:
    resp = await client.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

Outro trecho de código de [wikipics-demo](wikipics-demo.ipynb),
para comparação com o código usando `await` em seguida.

```python
url = get_sample_url(1_000_000)
img_rec = fetch_blocking(url)  # main thread blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)
```

In [3]:
url = get_sample_url(1_000_000)
async with AsyncClient() as client:
    img_rec = await fetch(client, url)  # console blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

     958_960 bytes | Beachy_Head_and_Lighthouse%2C_East_Sussex%2C_England_-_April_2010_crop_horizon_corrected.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe1\x0fmExif\x00\x00II*\x00\x…

Mais um trecho de [wikipics-demo](wikipics-demo.ipynb), para comparação:

```python
img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

for url in urls:
    img_rec = await fetch(url)  # SyntaxError: 'await' outside function
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')
```


Na próxima célula eu desisti porque não descobri como produzir os resultados de `download_pics` dentro do Jupyter.

Veja o código funcionando no script Python `wikipics-async-demo.py`.

In [4]:
import asyncio

img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

async def download_pics(urls):
    async with AsyncClient() as client:
        tasks = [fetch(client, url) for url in urls]
        results = await asyncio.gather(*tasks)
    return results

for img_rec in await download_pics(urls):
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')    

Image(value=b'\xff\xd8\xff\xe1\x16\x14Exif\x00\x00II*\x00\x08\x00\x00\x00\n\x00\x0f\x01\x02\x00\x06\x00\x00\x0…

   2_015_977 bytes | Litoria_infrafrenata_-_Julatten.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x02\x00&\x00&\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\…

   1_977_558 bytes | Frederic_Edwin_Church_-_Aurora_Borealis_-_Google_Art_Project.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x08\x95\x08\x95\x00\x00\xff\xe1\x13~Exif\x00\x00II*…

...

   2_019_740 bytes | Bald.eagle.closeup.arp-sh.750pix.jpg


Image(value=b'\xff\xd8\xff\xe1%\xfeExif\x00\x00II*\x00\x08\x00\x00\x00\t\x00\x0f\x01\x02\x00\x06\x00\x00\x00z\…

   1_975_417 bytes | Dolceacqua43_-_Artista_locale_mentre_dipinge_un_acquarello.jpg


Image(value=b'\xff\xd8\xff\xe1$\tExif\x00\x00II*\x00\x08\x00\x00\x00\x0c\x00\x0e\x01\x02\x00\xbb\x00\x00\x00\x…

   1_972_894 bytes | Barack_Obama_family_portrait_2011.jpg
TOTAL BYTES: 19_962_219


In [5]:
# %%time
# Não funciona:
#   File <timed exec>:18
# SyntaxError: 'await' outside function

from gallery import Gallery

num_images = 10

urls = get_sample_urls(15_000_000, num_images)
gallery = Gallery(num_images)
gallery.display()

async def display_pics(urls):
    async with AsyncClient() as client:
        tasks = [fetch(client, url) for url in urls]
        for i, coroutine in enumerate(asyncio.as_completed(tasks)):
            img_rec = await coroutine
            gallery.update(i, img_rec.pixels, img_rec.name)

await display_pics(urls)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)  14_541_999 bytes | Pahit-Pahit_Manis_pamphlet_%28obverse%29.jpg
( 2)  14_824_826 bytes | Mary_Coriolano2.jpg
( 3)  15_247_075 bytes | Exterior_of_Blenduk_Church%2C_Semarang%2C_2014-06-18.jpg
( 4)  14_975_113 bytes | Shoki2_detail.jpg
( 5)  15_031_716 bytes | Rembrandt_Harmensz._van_Rijn_-_Portret_van_een_paar_als_oudtestamentische_figuren%2C_genaamd_%27Het_Joodse_bruidje%27_-_Google_Art_Project.jpg
( 6)  15_062_885 bytes | Pahalgam_Valley.jpg
( 7)  14_784_640 bytes | Zaandam2.jpg
( 8)  15_461_969 bytes | First_Manassas_map2.jpg
( 9)  15_407_615 bytes | St_Etheldreda%27s_Church_Interior%2C_London%2C_UK_-_Diliff.jpg
(10)  15_508_311 bytes | Berlin_-_Schloss_Bellevue2.jpg
TOTAL BYTES: 150_846_149
